In [138]:
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/index.html
# sudo pip3 install jupyter datascience matplotlib pandas
# https://github.com/bear/python-twitter
# sudo pip3 install python-twitter

import twitter
# tweepy has potential as an alternative to python-twitter

api = twitter.Api(consumer_key='',
                  consumer_secret='',
                  access_token_key='',
                  access_token_secret='',
                  tweet_mode= 'extended',
                  sleep_on_rate_limit=True)
print(api.CheckRateLimit('https://api.twitter.com/1.1/help/privacy.json'))
# print(api.VerifyCredentials())

# statuses = api.GetUserTimeline(screen_name=user)

EndpointRateLimit(limit=15, remaining=15, reset=1525812326)


In [136]:
# wrapper around numpy and pandas
# https://github.com/data-8/datascience
from datascience import *
import json

## Twitter gov has several US lists for congress and appointees.
# https://twitter.com/TwitterGov/lists/us-governors/members
twittergov = [
    "us-governors",
    "us-senate",
    "us-house",
    "us-secretaries-of-state",
    "us-cabinet"
]

# List members and write them to CSV and JSON
for user_group in twittergov:
    list_members = api.GetListMembers(owner_screen_name="TwitterGov",slug=user_group)
    Table().with_columns([
        "ID", [u.id for u in list_members],
        "Screen Name", [u.screen_name for u in list_members]
    ]).to_csv("twittergov/" + user_group + ".csv")
    with open("twittergov/" + user_group + ".json", "w+") as f:
        for member in list_members:
            json.dump(member.AsDict(), f, ensure_ascii = False, sort_keys=True)
            f.write('\n')
    print("Members(" + user_group + "): " + str(len(list_members)))


Members(us-cabinet): 40


In [127]:
stategov = Table.read_table("twittergov/us-senate.csv")
print(stategov)

ID                 | Screen Name
983348251972816896 | SenHydeSmith
941080085121175552 | SenDougJones
941000686275387392 | SenTinaSmith
816683274076614656 | SenJohnKennedy
811313565760163844 | SenCortezMasto
803694179079458816 | SenKamalaHarris
2964174789         | SenThomTillis
2955485182         | SenatorRounds
2891210047         | SenDanSullivan
2863210809         | sendavidperdue
... (90 rows omitted)


In [137]:
import json
import datetime
import os

# Stop at January 1, 2017
jan2017 = datetime.datetime(2017,1,1).timestamp()


screen_names = Table.read_table("twittergov/us-cabinet.csv").column("Screen Name")
out_folder = "twittergov/us-cabinet"

# Write each account to a file.
for user_id in screen_names:
    out_file= out_folder + "/" + str(user_id) + ".json"
    # Per twitter API, 200 per fetch, 3200 total
    batch_size = 200
    max_batches = 16
    max_id = 0
    print(user_id)
    with open(out_file, "w+") as f:
        for x in range(0, max_batches):
            twit = api.GetUserTimeline(screen_name=user_id,count=batch_size,include_rts=False,max_id=max_id,trim_user=True)
            # print("Results: " + str(len(twit)))
            if len(twit) == 0: break
            for status in twit:
                json.dump(status.AsDict(), f, ensure_ascii = False, sort_keys=True)
                f.write('\n')
            last_twit = twit[-1]
            if last_twit.created_at_in_seconds < jan2017: break
            max_id = last_twit.id - 1

SecPompeo
SecAzar
SecNielsen
MickMulvaneyOMB
USTradeRep
SecretarySonny
SecretaryRoss
OMBPress
EPAScottPruitt
SecretaryPerry
BetsyDeVosED
SecretaryCarson
SecretaryZinke
SecElaineChao
WhiteHouse
SBALinda
SecretaryAcosta
Cabinet
VP
stevenmnuchin1
CIA
USDOT
USUN
ENERGY
SBAgov
USTreasury
CommerceGov
DeptVetAffairs
Interior
TheJusticeDept
DeptofDefense
USDA
HHSGov
nikkihaley
usedgov
USDOL
HUDgov
DHSgov
EPA
StateDept
